# 1 Initialize notebook

## 1.1 Import libraries

In [ ]:
import copy
import glob
import itertools
import os
import pickle
from pprint import pprint
import re
import yaml
import warnings

from datetime import date

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import ndimage, spatial, stats
import seaborn as sns
import trackpy as tp

pd.options.mode.chained_assignment = None

In [ ]:
from data_collections import *
from image_analysis import *
from event_detection import *

In [ ]:
from IPython.display import display, Markdown, Latex
from tqdm.notebook import tqdm

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib as mpl

## 1.2 Set global parameters

In [ ]:
# Set style options for graphs
mpl.rc('font', size=9)
mpl.rc('axes', titlesize=9)
mpl.rc('font', family='sans-serif') 
mpl.rc('font', serif='Arial')

cm = 1 / 2.54  # centimeters in inches

In [ ]:
# Define colors used in manuscript
red = '#CE214C'
orange = '#F36F32'
gold = '#F0BE08'
green = '#16A970'
cyan = '#13B4E1'
blue = '#1B1B87'
magenta = '#BB178A'
white = '#FFFFFF'
gray = '#BBBBBB'
black = '#000000'

In [ ]:
today = str(date.today())

## 1.3 Define functions

In [ ]:
def get_ci(array, confidence=0.95):
    """
    Create confidence interval for population mean weight (default = 95%).
    """
    return stats.t.interval(
        confidence=confidence,
        df=len(array)-1,
        loc=np.mean(array),
        scale=stats.sem(array),
    )

In [ ]:
def get_group(df, by, on):
    """
    Helper function to get DataFrame group.
    """
    return df[df[by] == on]

def apply(dict_in, func, keys=None, copy=False):
    """
    Helper function to apply function to dict
    """
    if keys is None:
        keys = dict_in.keys()
    
    if copy:
        dict_in = copy.deepcopy(dict_in)
    
    result = {key : func(dict_in[key]) for key in keys}
        
    return result

In [ ]:
def merge_data_dicts(dfs_dict, channels, df_types=None, rounding=5, intensity='raw'):
    """
    For a given experiment, merge requested data dictionaries together.
    """
    
    ndim = 3
    axis_names = 'z', 'y', 'x'
    px2um = {'z' : 0.210, 'y' : 0.104, 'x' : 0.104}
    
    #dfs_dict = self._data
    if df_types is None:
        df_types = dfs_dict.keys()
    #channels = dfs_dict[list(df_types)[0]].keys()
    
    dfs_merged = dict()
    for channel in channels:
        # Merge on xyzt data for each particle (in px)
        merge_columns = ['frame', 'z_px', 'y_px', 'x_px']

        # Assume that blobs and intensities are available
        df1 = dfs_dict['blobs'][channel].round(rounding)
        df2 = dfs_dict['intensities'][channel].round(rounding)

        # Merge blobs and intensities data
        df = pd.merge(df1, df2, on=merge_columns)
        
        # Drop duplicates and sort data
        df = df.drop_duplicates(subset=['z_px', 'y_px', 'x_px', 'frame'])
        
        for df_type in df_types:
            try:
                df3 = dfs_dict[df_type][channel].round(rounding)
                if df_type in ('blobs', 'intensities'):
                    # Skip previously merged data
                    continue
                elif df_type in ('tracked', ):
                    # Rename tracked columns
                    rename_columns = {'x' : 'x_um', 'y' : 'y_um', 'z' : 'z_um'}
                    df3.rename(columns=rename_columns, inplace=True)

                    # Merge tracked and intensities/blobs data on xyzt data (in um)
                    merge_columns = ['frame', 'z_um', 'y_um', 'x_um']
                    kep_columns = ['frame', 'z_um', 'y_um', 'x_um', 'track', 'segment', 'parent']
                    df = pd.merge(df, df3[kep_columns], on=merge_columns)

                    # Drop duplicates and sort data
                    df = df.drop_duplicates(subset=['track', 'frame'])
                    df = df.sort_values(by=['track', 'frame'])
                else:
                    # Merge based on frame and label
                    df = pd.merge(df, df3, on=['frame', 'labels'])

                    # Sort data
                    df = df.sort_values(by=['frame', 'labels'])
            except Exception as e:
                print(e)
        
        # Clean up the merged DataFrame
        df = df.dropna()

        # Make a new column for the average radius (in pixels)
        sigma_px = df.filter(regex='sigma').mean(axis=1)
        df['r_px'] = sigma_px * np.sqrt(ndim)

        # Make a new column for the average radius (in microns), assuming correct lateral dimensions
        sigmas_um = df[['sigma_y', 'sigma_x']] * [px2um['y'], px2um['x']]
        df['r_um'] = (sigmas_um * np.sqrt(ndim)).mean(axis=1)

        # Save the merged DataFrame
        dfs_merged[channel] = df
    
    return dfs_merged

# 2 Import new datasets (_run once_)

## 2.1 Set import parameters 

In [ ]:
# Set analysis directory
top_dir = '.'

# Set condition for analysis
condition = 'RPE1 WT'

# Set experiment search pattern
experiment_pattern = 'APPL1EEA1'

# Predefine channels
channel1, channel2 = channels = 'APPL1', 'EEA1'

# Predefine data types each corresponding to set of CSV files for each channel
data_types = 'tracked', 'collisions', 'conversions', 'intensities', 'blobs'

In [ ]:
# Set specific column names
track_columns = ['track_' + channel for channel in channels]
id_columns = track_columns + ['movie']

In [ ]:
# Define channel-specific colors
colors = {
    channel1 : cyan,
    channel2 : magenta,
}

## 2.2 Load data from CSV files

In [ ]:
# Load analysis CSV files
path_lists = list()
data_dicts = list()

condition_dir = os.path.join(top_dir, 'Tracking', condition)
experiment_dirs = glob.glob(os.path.join(condition_dir, experiment_pattern))

for experiment_dir in experiment_dirs:
    if not os.path.isdir(experiment_dir): continue
    cell_dirs = glob.glob(os.path.join(experiment_dir, '*'))
    for cell_dir in cell_dirs:
        if not os.path.isdir(cell_dir):
            continue
        print(cell_dir)
        
        csv_data = dict()
        for data_type in data_types:
            csv_data[data_type] = dict()
            for channel in channels:
                csv_path = os.path.join(cell_dir, data_type.title() + '_' + channel + '.csv')
                try:
                    csv_data[data_type][channel] = pd.read_csv(csv_path)
                except FileNotFoundError:
                    pass

        path_lists.append(cell_dir)
        data_dicts.append(csv_data)

## 2.3 Check data based on conversions identified

In [ ]:
print(condition.upper())
for cell_index, data_dict in enumerate(data_dicts):
    # Attempt to read relevant data dicts
    try:
        tracked = data_dict['tracked'][channel1]
        conversions = data_dict['conversions'][channel1].drop_duplicates(subset=track_columns)
    except KeyError:
        continue

    # Read out number of possible and likely conversions
    convert_lens = (conversions[conversions['conversion'] == True]
                    .filter(like='overlap')
                    .diff(axis=1)
                    ['overlap_stop']
                    .values
                   )
    convert_keep = convert_lens > 4
    overlap_count = len(convert_lens)
    convert_count = len(convert_lens[convert_keep])

    # Collect basis statistics
    if overlap_count > 0:
        convert_pct = 100 * convert_count / overlap_count
    else:
        convert_pct = 0

    # Print results to screen
    cell_name = ', '.join(path_lists[cell_index].split(os.sep)[-2:])
    message = f"  {cell_name} : {convert_count:d} likely conversions ({convert_pct:0.1f}%)"
    if convert_pct > 0:
        message += f"; median length = {np.median(convert_lens[convert_keep])}"
        message += f"; max length = {np.max(convert_lens)}"

    print(message)
print()

## 2.4 Combine data from each condition together

In [ ]:
for data_type in data_types:
    data_dfs = dict()
    for channel in channels:
        data_dfs[channel] = list()
        for data_dict, path_list in zip(data_dicts, path_lists):
            condition, experiment, cell_name = path_list.split(os.sep)[-3:]
            if data_type == 'tracked':
                data_dict[data_type] = merge_data_dicts(data_dict, channels, ('blobs', 'tracked', 'intensities'))
                
            if channel in data_dict[data_type]:
                data_df = data_dict[data_type][channel]
                if condition == '_Senthil':
                    data_df['condition'] = 'RPE1 WT'
                    data_df['movie'] = int(cell_name.replace('cell', ''))
                else:
                    data_df['movie'] = cell_name
                data_dfs[channel].append(data_df)
        if channel in data_dfs and len(data_dfs[channel]) > 0:
            data_dfs[channel] = pd.concat(data_dfs[channel])
    
    if data_type == 'tracked':
        tracked1 = data_dfs[channel1]
        tracked2 = data_dfs[channel2]
    elif data_type == 'collisions':
        collisions = data_dfs[channel1]
    elif data_type == 'conversions':
        conversions = data_dfs[channel1]

collisions.drop_duplicates(ignore_index=True, inplace=True)
conversions.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
# Get frame rates of each movie
DEFAULT_FRAME_RATE = 2.6
movie_list = tuple(tracked1['movie'].unique())
frame_rates = dict.fromkeys(movie_list)

for movie in movie_list:
    try:
        match = re.search(r'([\d.]*)spv', movie)
        if match:
            frame_rate = float(match.group(1))
        else:
            print(movie)
            frame_rate = DEFAULT_FRAME_RATE
    except:
        frame_rate = DEFAULT_FRAME_RATE
    frame_rates[movie] = frame_rate

In [ ]:
# Convert data from each movie into a DataFrame
data1 = list()
data2 = list()
buckets = list()

for movie in movie_list:
    event = 0
    
    # Get all conversions for the current movie
    df = conversions.groupby('movie').get_group(movie)
    
    # Get all frames for the current movie
    frames = tracked1.groupby('movie').get_group(movie)['frame'].unique()

    # Remove tracks starting and ending with the movie
    interior = (df['overlap_start'] > frames.min()) & (df['overlap_stop'] < frames.max())
    df = df[interior]
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        event += 1
        
        track1, track2 = row[track_columns]
        t1 = tracked1.groupby(['movie', 'track']).get_group((movie, track1))
        t2 = tracked2.groupby(['movie', 'track']).get_group((movie, track2))
        
        # Save data results for APPL1
        df1 = t1.filter(like='signal')
        try:
            df1 = df1.apply(lambda y : signal.savgol_filter(y, 3, 1))
        except:
            pass
        
        df1['frame_abs'] = t1['frame']
        df1['time_abs'] = df1['frame_abs'] * frame_rate
        df1['frame'] = t1['frame'] - row['overlap_start']
        df1['time'] = df1['frame'] * frame_rate
        df1['length'] = row['overlap_stop'] - row['overlap_start'] + 1
        df1['conversion'] = row['conversion']
        df1['event'] = event
        df1['movie'] = movie
        df1['track_'+channel1] = row['track_'+channel1]
        df1['track_'+channel2] = row['track_'+channel2]
        df1['r_um'] = t1['r_um']
        df1['V_um'] = (4 * np.pi / 3) * df1['r_um'] ** 3
        
        # Save data results for EEA1
        df2 = t2.filter(like='signal')
        try:
            df2 = df2.apply(lambda y : signal.savgol_filter(y, 3, 1))
        except:
            pass
        
        df2['frame_abs'] = t2['frame']
        df2['time_abs'] = df2['frame_abs'] * frame_rate
        df2['frame'] = t2['frame'] - row['overlap_start']
        df2['time'] = df2['frame'] * frame_rate
        df2['length'] = row['overlap_stop'] - row['overlap_start'] + 1
        df2['conversion'] = row['conversion']
        df2['event'] = event
        df2['movie'] = movie
        df2['track_'+channel1] = row['track_'+channel1]
        df2['track_'+channel2] = row['track_'+channel2]
        df2['r_um'] = t2['r_um']
        df2['V_um'] = (4 * np.pi / 3) * df2['r_um'] ** 3
        
        
        # Filter based on decrease in APPL1 / increased in EEA1
        start1, start2 = (df1[(df1['frame'] >= 0) & 
                              (df1['frame'] < df1['length'] / 5)]
                           .filter(like='sub').mean(axis=0)[['signal_sub_'+channel1, 'signal_sub_'+channel2]].values)
        stop1, stop2 = (df1[(df1['frame'] < df1['length']) & 
                            (df1['frame'] >= df1['length'] / 5)]
                         .filter(like='sub').mean(axis=0)[['signal_sub_'+channel1, 'signal_sub_'+channel2]].values)
        
        # Save threshold criteria for filtering results
        bucket = {'track_'+channel1 : track1, 
                  'track_'+channel2 : track2, 
                  'movie' : movie, 'event' : event}
        bucket['tracked_during_'+channel1] = row['overlap_start'] in t1['frame'].values
        bucket['tracked_during_'+channel2] = row['overlap_start'] in t2['frame'].values
        bucket['decrease_during_'+channel1] = stop1 < start1
        
        bucket['increase_during_'+channel2] = stop2 > start2
        bucket['started_before_'+channel1] = row['track_'+channel1+'_start'] < row['overlap_start']
        bucket['started_during_'+channel2] = row['track_'+channel2+'_start'] >= row['overlap_start']
        bucket['stopped_during_'+channel1] = row['track_'+channel1+'_stop'] <= row['overlap_stop']
        bucket['continued_after_'+channel2] = row['track_'+channel2+'_stop'] > row['overlap_stop']
        
        buckets.append(bucket)
        
        data1.append(df1)
        data2.append(df2)

buckets = pd.DataFrame(buckets)
data1 = pd.concat(data1, ignore_index=True)
data2 = pd.concat(data2, ignore_index=True)

In [ ]:
# Recalculate tracks and movies
tracks1, tracks2, movies = map(np.asarray, zip(*conversions[id_columns].values))

## 2.5 Save intermediate results

In [ ]:
# Intermediate results will save to the following file
csv_name = f"Dataset_{'+'.join(channels)}_{today}.csv"

In [ ]:
# Gather all data into a dict
datasets = {
    'Metadata' : {
        'colors' : colors,
        'channels' : channels,
        'movie_list' : movie_list,
        'frame_rates' : frame_rates,
    },
    'Filter1' : {
        'tracks1' : tracks1,
        'tracks2' : tracks2,
        'movies' : movies,
        'tracked1' : tracked1,
        'tracked2' : tracked2,
        'collisions' : collisions,
        'conversions' : conversions,
        'buckets': buckets,
        'data1' : data1,
        'data2' : data2,
    },
}

with open(csv_name, 'wb') as f:
    f.write(pickle.dumps(datasets))

# 3 Check basic dataset statistics

## 3.1 Import previous datasets

In [ ]:
# Get proper CSV name (latest export by default)
csv_index = -1
csv_name = sorted(glob.glob('Dataset*.csv'))[csv_index]

In [ ]:
with open(csv_name, 'rb') as f:
    datasets = pickle.load(f)

# Gather all data into a dict
channel1, channel2 = channels = datasets['Metadata']['channels']
colors = datasets['Metadata']['colors']
movie_list = datasets['Metadata']['movie_list']
frame_rates = datasets['Metadata']['frame_rates']

tracks1 = copy.deepcopy(datasets['Filter1']['tracks1'])
tracks2 = copy.deepcopy(datasets['Filter1']['tracks2'])
movies = copy.deepcopy(datasets['Filter1']['movies'])

tracked1 = copy.deepcopy(datasets['Filter1']['tracked1'])
tracked2 = copy.deepcopy(datasets['Filter1']['tracked2'])
collisions = copy.deepcopy(datasets['Filter1']['collisions'])
conversions = copy.deepcopy(datasets['Filter1']['conversions'])

buckets = copy.deepcopy(datasets['Filter1']['buckets'])
data1 = copy.deepcopy(datasets['Filter1']['data1'])
data2 = copy.deepcopy(datasets['Filter1']['data2'])

In [ ]:
# Set specific column names
track_columns = ['track_' + channel for channel in channels]
id_columns = track_columns + ['movie']

## 3.2 Tracked particle lengths

In [ ]:
data_type = 'Tracked'
scale = 'linear'

fig, ax = plt.subplots(figsize=(8, 5))
for channel, df in zip(channels, (tracked1, tracked2)):
    hist, bins = np.histogram(np.bincount(df['track'].astype(int)), bins=np.arange(df['frame'].min(), df['frame'].max() + 1), density=True)
    ax.step(bins[1:], hist[0:], color=colors[channel], label=channel)
ax.set_ylabel('relative frequency')
ax.set_xlabel('track length (frames)')
ax.set_xlim(1e0, 1e+2)
ax.set_ylim(1e-6, 1e-1)
ax.set_xscale(scale)
ax.set_yscale(scale)
ax.legend()
fig.tight_layout(pad=3.0)

## 3.3 Step lengths

In [ ]:
tracks = tracked2['track'].unique()
disps = list()
for track in tracks:
    t = get_group(tracked1, 'track', track)
    if len(t) <= 10: continue
    d = np.diff(t[['x_um', 'y_um', 'z_um']].values, axis=0)
    z = np.cumsum(np.sum(np.abs(d), axis=1))
    disps.append(z)

disp_lens = [len(disp) for disp in disps]

In [ ]:
fig = plt.figure(figsize=(13, 13))
ax = plt.axes(projection='3d')

indexes = np.argsort(disp_lens)[:-1]
n = len(indexes)
for index in indexes:
    xs = disps[index]
    ys = disp_lens[index] * np.ones_like(xs) / n
    zs = np.arange(len(xs))
    
    ax.plot(xs, ys, zs, linestyle='-', alpha=0.25)
    ax.set_xlabel(u"3D displacement ($\mu$m)")
    ax.set_ylabel(u"fraction of traces")
    ax.set_zlabel(u"frames (#)")

ax.view_init(elev=30, azim=-60)

In [ ]:
# Calculate step size statistics
steps = dict.fromkeys(channels)
for channel, df in zip(channels, (tracked1, tracked2)):
    disps = tp.motion.relate_frames(df.rename(columns={'t' : 'frame', 'track' : 'particle'}), 0, 1, \
                                    pos_columns=['x_um', 'y_um', 'z_um'])[['dx_um', 'dy_um', 'dz_um']].dropna()
    steps[channel] = np.abs(disps)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 5))
for index, channel in enumerate(channels):
    ax = axs[index]
    for axis, color in zip(
        ('dx_um', 'dy_um', 'dz_um'),
        ('brown', 'crimson', 'orange'),
    ):
        ax.hist(steps[channel][axis], density=True, color=color, histtype='step', label=axis, linewidth=1.5)
        ax.set_xlabel('inter-frame displacement ($\mu$m)')
        ax.set_ylabel('probability density')
        ax.set_xlim(0, 100)
        ax.legend()
        ax.set_title(channel)
fig.tight_layout(pad=3.0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
for index, channel in enumerate(channels):
    ax.hist(steps[channel].mean(axis=1), density=True, color=colors[channel], alpha=0.5)
    ax.set_xlabel('inter-frame displacement ($\mu$m)')
    ax.set_ylabel('probability density')
    ax.set_xlim(0, 25)
ax.legend(channels)
fig.tight_layout(pad=3.0)

In [ ]:
axes = 'x_um', 'y_um', 'z_um'
for channel, df in zip(channels, (tracked1, tracked2)):
    print(channel, {axis : round(df[axis].max() - df[axis].min()) for axis in axes})

## 3.4 Conversion statistics

In [ ]:
params = {'x' : 'convert_len', 'bins' : np.arange(-0.5, 15, 1), 'fill' : True, 'stat' : 'density', 'cumulative' : False}
fig, ax = plt.subplots(figsize=(8, 5))
data = conversions.drop_duplicates(subset='track_Lamp1')
data = data[data['conversion']]
data['convert_len'] = data['overlap_stop'] - data['overlap_start']
sns.histplot(ax=ax, data=data, **params, color='teal')
ax.set_xlim(0, 15)
ax.set_ylim(0, 0.8)
fig.tight_layout(pad=3.0)

In [ ]:
params = {'x' : 'convert_len', 'bins' : np.arange(-0.5, 15, 1), 'fill' : True, 'stat' : 'count', 'cumulative' : False}
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(ax=ax, data=data, **params, color='cornflowerblue')
ax.set_xlim(0, 15)
fig.tight_layout(pad=3.0)

In [ ]:
def get_num_frames(conversion, trajectory):
    track_start, track_stop = min(trajectory[channel1]['frame']), max(trajectory[channel2]['frame'])
    convert_start, convert_stop = tuple(conversion.iloc[0][['convert_start', 'convert_stop']])
    
    before = int(convert_start - track_start)
    during = int(convert_stop + 1 - convert_start)
    after = int(track_stop - convert_stop)
    total = int(track_stop + 1 - track_start)
    
    return {'before' : before, 'during' : during, 'after' : after, 'total' : total}

In [ ]:
def get_trajectory(conversions, collisions, trackA):
    conversion = get_group(conversions, 'track', trackA)
    collisionA = get_group(collisions[channel1], 'track', trackA)
    trackE = conversion['collide_track'][conversion['frame'] == conversion['convert_stop']].iloc[-1]
    collisionE = get_group(collisions[channel2], 'track', trackE)
    return {channel1 : collisionA, channel2 : collisionE}

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
frames = np.sort(collisions['frame'].unique())
sns.scatterplot(ax=ax, data=data, x='overlap_start', y='convert_len', alpha=0.05, color='crimson', label='convert_start')
sns.scatterplot(ax=ax, data=data, x='overlap_stop', y='convert_len', alpha=0.05, color='orange', label='convert_stop')
ax.set_xlim(min(frames), max(frames))
ax.set_ylim(0, 50)
ax.legend()
fig.tight_layout(pad=3.0)

# 4 Filter dataset by relative timing of events

## 4.1 Create event-based filters

In [ ]:
# Get filters from buckets
filters = buckets[buckets.columns[4:]]
filters.sum() / len(filters)

In [ ]:
# Create heatmap to visualize fractions that passed each filter
heatmap = pd.DataFrame(data=0, index=filters.columns, columns=filters.columns)
for row, col in itertools.product(heatmap.columns, heatmap.columns):
    heatmap.at[row, col] = sum(filters[row] & filters[col])

# Visualize the heatmap
fig, ax = plt.subplots(figsize=(8, 5))
cmap = LinearSegmentedColormap.from_list('custom', [(0, white), (0.25, gray), (0.5, gold), (0.75, magenta), (1, black)])
sns.heatmap(heatmap / len(buckets), annot=True, cmap=cmap, fmt='0.2f', linewidths=2, square=True, ax=ax)

In [ ]:
# Create code to display decision tree in flowchart.js
df0 = copy.deepcopy(filters)
df1 = df0[df0['tracked_during_'+channel1] & df0['tracked_during_'+channel2]]
df2a = df1[df1['started_before_'+channel1]]
df2b = df1[df1['continued_after_'+channel2]]
df3 = df1[df1['started_before_'+channel1] & df1['continued_after_'+channel2]]
df4a = df3[df3['stopped_during_'+channel1]]
df4b = df3[df3['decrease_during_'+channel1]]
df4c = df3[df3['increase_during_'+channel2]]
df5 = df3[df3['stopped_during_'+channel1] & df3['decrease_during_'+channel1] & df3['increase_during_'+channel2]]
df7 = df5[df5['started_during_'+channel2]]
df8 = df7

flowchart = f"""
st1=>start: initial set of potential conversions
N = {len(df1)} ({100*len(df1)/len(df1):0.1f}% of initial dataset)
op2=>operation: {channel1} track started before cotracking
N = {len(df2a)} ({100*len(df2a)/len(df1):0.1f}% of initial dataset)
op3=>operation: {channel2} track continued after cotracking
N = {len(df2b)} ({100*len(df2b)/len(df1):0.1f}% of initial dataset)
op3=>operation: {channel2} track continued after cotracking
N = {len(df3)} ({100*len(df3)/len(df1):0.1f}% of initial dataset)
op4=>operation: {channel1} track stopped during cotracking
N = {len(df4a)} ({100*len(df4a)/len(df3):0.1f}% of filtered dataset)
op5=>operation: {channel1} intensity decreased during cotracking
N = {len(df4b)} ({100*len(df4b)/len(df3):0.1f}% of filtered dataset)
op6=>operation: {channel2} intensity increased during cotracking
N = {len(df4c)} ({100*len(df4c)/len(df3):0.1f}% of filtered dataset)
op6=>operation: {channel2} intensity increased during cotracking
N = {len(df5)} ({100*len(df5)/len(df3):0.1f}% of filtered dataset)
cond=>condition: filter out fusions
op7=>operation: {channel2} track started during cotracking
N = {len(df7)} ({100*len(df7)/len(df5):0.1f}% of previous step)
en1=>end: final set of filtered conversions
N = {len(df5)} ({100*len(df5)/len(df1):0.1f}% of initial dataset)
en2=>end: final set of filtered conversions
N = {len(df8)} ({100*len(df8)/len(df1):0.1f}% of initial dataset)

st1->op2
op2->op3
op3->op4
op4->op5
op5->op6
op6->cond
cond(no)->en1
cond(yes)->op7
op7->en2
"""
print("http://flowchart.js.org/")
print(flowchart)

## 4.2 Filter data based on provided criteria

In [ ]:
# Filter out any lines not to be filtered on
keep = (
    buckets['tracked_during_'+channel1] & 
    buckets['tracked_during_'+channel2] &
    buckets['decrease_during_'+channel1] & 
    buckets['increase_during_'+channel2] & 
    buckets['started_before_'+channel1] & 
    buckets['stopped_during_'+channel1] & 
    buckets['continued_after_'+channel2]
)

tracks1, tracks2, movies = zip(*buckets[keep][id_columns].drop_duplicates().values)
ids_keep = tracks1, tracks2, movies
print(f"There are {len(tracks1)} tracks remaining.")

In [ ]:
# Filter down datasets according to filter criteria
data1 = data1[data1['track_'+channel1].isin(tracks1) & data1['movie'].isin(movies)]
data2 = data2[data2['track_'+channel2].isin(tracks2) & data2['movie'].isin(movies)]

# The following line should evaluate to True
all(data2 == data2.query(f"event in {list(buckets['event'].values)}"))

In [ ]:
# Filter conversions by same criteria
conversions2 = list()
for track1, track2, movie in tqdm(zip(*ids_keep), total=len(tracks1)):
    conversions2.append(conversions.groupby(['track_'+channel1, 'track_'+channel2, 'movie']).get_group((track1, track2, movie)))
conversions2 = pd.concat(conversions2, ignore_index=True)

# Sort and drop duplicates
conversions2 = conversions2.sort_values(by='overlap_start')
conversions2.drop_duplicates(subset=['track_'+channel1, 'movie'], ignore_index=True, inplace=True, keep='last')

In [ ]:
# Recalculate tracks and movies
tracks1, tracks2, movies = map(np.asarray, zip(*conversions2[id_columns].values))

In [ ]:
# Remove all collisions also classified as cotracking events
collisions2 = collisions.drop_duplicates(ignore_index=True)
collisions2 = collisions2[collisions2['movie'].isin(set(np.unique(movies)))]
cotrackings = conversions[conversions['movie'].isin(set(np.unique(movies)))]
for _, row in tqdm(cotrackings.iterrows(), total=len(cotrackings)):
    track1, track2, movie = row[id_columns]
    frame1, frame2 = row[['overlap_start', 'overlap_stop']]
    try:
        df12 = collisions2.groupby(id_columns).get_group((track1, track2, movie))
    except KeyError:
        continue
    ma12 = df12[df12['frame'].map(lambda frame: (frame >= frame1) and (frame <= frame2))].index
    collisions2.drop(ma12, inplace=True)
collisions2.reset_index(drop=True, inplace=True)

print(f"{len(collisions2)} remaining ({100 * len(collisions2) / len(collisions):0.2f}%)")

## 4.3 Save intermediate results

In [ ]:
# Save filtered results to file
datasets['Filter2'] = {
    'tracks1' : tracks1,
    'tracks2' : tracks2,
    'movies' : movies,
    'tracked1' : tracked1,
    'tracked2' : tracked2,
    'collisions' : collisions2,
    'conversions' : conversions2,
    'buckets': buckets,
    'data1' : data1,
    'data2' : data2,
}

with open(csv_name, 'wb') as f:
    f.write(pickle.dumps(datasets))

## 4.4 View individual time series

In [ ]:
# Collate all trajectories for visualization
meaned = True
normed = True

signals = dict.fromkeys(channels)
for channel, data in zip(channels, (data1, data2)):
    for (movie, event), grouped in data.groupby(['movie', 'event']):
        if 0 not in grouped['frame'].values:
            continue
        
        ints = grouped.filter(like='signal')
        if meaned:
            ints = ints.div(grouped['V_um'].values, axis=0)
        if normed:
            ints = ints.div(ints[(grouped['frame'] == 0)].values, axis=1)
        grouped[ints.columns] = ints
        
        signals[channel] = pd.concat([signals[channel], grouped], ignore_index=True)

In [ ]:
int_type = 'sub'
min_length = 0
max_length = 250

fig, axss = plt.subplots(2, 2, figsize=(8, 8))
for axs, c1 in zip(axss, channels):
    data = copy.deepcopy(signals[c1])
    data = data[(data['length'] >= min_length) & (data['length'] < max_length)]
    for (movie, event), df in data.groupby(['movie', 'event']):
        for ax, c2 in zip(axs, channels):
            columns = ['time', 'signal_' + int_type + '_' + c2]
            x, y = map(np.asarray, zip(*df.sort_values(by='time')[columns].values))
            
            if len(x) < min_length:
                continue
            
            if normed and np.any(np.abs(y) > 2.5):
                continue
            
            ax.plot(x, y, alpha=0.01, color=colors[c2])

for axs, c1 in zip(axss, channels):
    for ax, c2 in zip(axs, channels):
        ax.set_xlim(-40, +40)
        if normed:
            ylim = 0, 3
        else:
            if c2 == channel1:
                ylim = 0, 25000
            else:
                ylim = 0, 50000
            if c1 == channel2:
                ylim = ylim[0], ylim[1] * 2
        ax.set_ylim(*ylim)    
        ax.set_xlabel('Time from Start of Cotracking (s)')
        ax.set_ylabel('Intensity')
        ax.set_title(f"{c1} tracking, {c2} detection")

fig.tight_layout()

In [ ]:
# Preview results of Savitsky-Golay filter
fig, ax = plt.subplots(figsize=(5, 2))
ax.plot(x, y, cyan, label='raw data')
ax.plot(x, signal.savgol_filter(y, 3, 1), magenta, label='smoothed')
ax.set_xlabel('Time from Start of Cotracking (s)')
ax.set_ylabel('Intensity')
ax.legend(loc='upper right')

# 5 Filter collisions from cotracking events

## 5.1 Load intermediate results

In [ ]:
with open(csv_name, 'rb') as f:
    datasets = pickle.load(f)

# Gather all data into a dict
channel1, channel2 = channels = datasets['Metadata']['channels']
colors = datasets['Metadata']['colors']
movie_list = datasets['Metadata']['movie_list']
frame_rates = datasets['Metadata']['frame_rates']

tracks1 = copy.deepcopy(datasets['Filter2']['tracks1'])
tracks2 = copy.deepcopy(datasets['Filter2']['tracks2'])
movies = copy.deepcopy(datasets['Filter2']['movies'])

tracked1 = copy.deepcopy(datasets['Filter2']['tracked1'])
tracked2 = copy.deepcopy(datasets['Filter2']['tracked2'])
collisions2 = copy.deepcopy(datasets['Filter2']['collisions'])
conversions2 = copy.deepcopy(datasets['Filter2']['conversions'])

buckets = copy.deepcopy(datasets['Filter2']['buckets'])
data1 = copy.deepcopy(datasets['Filter2']['data1'])
data2 = copy.deepcopy(datasets['Filter2']['data2'])

## 5.2 Set thresholds of furthest distance for collisions

In [ ]:
# Save various pieces of collision data based on points of furthest distance,
# to remove any particles that may be cotracking and not actually colliding
collisions3_dict = dict()

# For references, store unfiltered data
collisions3_dict[0] = collisions2

In [ ]:
# Remove all collisions where EEA1 does not spend time away from APPL1
@dask.delayed
def filter_by_furthest_distance(row, tracked1, tracked2, threshold=1.0):
    track1, track2, movie = row[1][id_columns]
    
    # Get groups of colliding tracks
    df1 = (tracked1
           .groupby(['track', 'movie'])
           .get_group((track1, movie))
           .set_index('frame')
           .filter(like='_um')
          )
    df2 = (tracked2
           .groupby(['track', 'movie'])
           .get_group((track2, movie))
           .set_index('frame')
           .filter(like='_um')
          )
    
    # Reindex channel-specific dataframes to match
    index12 = df1.index.intersection(df2.index)
    df1 = df1.reindex(index12)
    df2 = df2.reindex(index12)

    # Get absolute positions in each channel
    pos1 = df1[['z_um', 'y_um', 'x_um']]
    pos2 = df2[['z_um', 'y_um', 'x_um']]
    
    # Get particle size (radius) in each channel
    rad1 = df1['r_um']
    rad2 = df2['r_um']
    
    # Calculate Euclidean distance between centroids
    com_dist = (pos1 - pos2).apply(np.linalg.norm, axis=1)
    
    # Calculate distance between surfaces
    surf_dist = com_dist - (rad1 + rad2)
    
    result = row[0]
    for value in surf_dist.values:
        if value > threshold:
            result = None
            break
    
    return result

In [ ]:
# Set the threshold for detection of collisions based on furthest distance (microns)
thresholds = 0.25, 0.5, 1.0

for threshold in thresholds:
# Execute the tasks in parallel
    args = tracked1, tracked2
    kwargs = dict(threshold=threshold)
    tasks = [filter_by_furthest_distance(row, *args, **kwargs) for row in collisions.iterrows()]
    with ProgressBar():
        row_indexes = [result for result in dask.compute(*tasks) if result is not None]

    # Remove all collisions where EEA1 does not spend time away from APPL1
    collisions3 = collisions.drop(row_indexes).reset_index(drop=True)
    collisions3_dict[threshold] = collisions3
    print(f"{threshold}: {len(collisions3)} remaining ({100 * len(collisions3) / len(collisions):0.2f}%)")

## 5.3 Save intermediate results

In [ ]:
# Save filtered results to file
datasets['Filter3'] = {
    'collisions_dict' : collisions3_dict,
}

with open(csv_name, 'wb') as f:
    f.write(pickle.dumps(datasets))

# 6 Filter collisions from near misses

## 6.1 Load intermediate results

In [ ]:
with open(csv_name, 'rb') as f:
    datasets = pickle.load(f)

# Gather all data into a dict
channel1, channel2 = channels = datasets['Metadata']['channels']
colors = datasets['Metadata']['colors']
movie_list = datasets['Metadata']['movie_list']
frame_rates = datasets['Metadata']['frame_rates']

tracks1 = copy.deepcopy(datasets['Filter2']['tracks1'])
tracks2 = copy.deepcopy(datasets['Filter2']['tracks2'])
movies = copy.deepcopy(datasets['Filter2']['movies'])

tracked1 = copy.deepcopy(datasets['Filter2']['tracked1'])
tracked2 = copy.deepcopy(datasets['Filter2']['tracked2'])
collisions2 = copy.deepcopy(datasets['Filter2']['collisions'])
conversions2 = copy.deepcopy(datasets['Filter2']['conversions'])

buckets = copy.deepcopy(datasets['Filter2']['buckets'])
data1 = copy.deepcopy(datasets['Filter2']['data1'])
data2 = copy.deepcopy(datasets['Filter2']['data2'])

## 6.2 Set thresholds of furthest distance for collisions

In [ ]:
# Remove any collisions where endosomes separater by greater than maximum distance
def collisions4_func(threshold):
    collisions = copy.deepcopy(datasets['Filter1']['collisions'])
    return collisions[collisions['surf_dist'] < threshold]

for threshold in np.arange(-0.4, +0.4, 0.1):
    # Remove all collisions where EEA1 and APPl1 are not close enough
    collisions4 = collisions4_func(threshold)
    print(f"{threshold:+0.1f}: {len(collisions4)} remaining ({100 * len(collisions4) / len(collisions):0.2f}%)")

## 6.3 Remove cases where EEA1 may be larger than expected

In [ ]:
# Merge collisions data with size data for APPL1
collisions5 = copy.deepcopy(datasets['Filter1']['collisions'])
collisions5 = (
    pd.merge(collisions5, tracked1[['r_um', 'track']], 
             left_on='track_APPL1', right_on='track',
            )
    .drop('track', axis=1)
    .rename(columns={'r_um' : 'r_um_APPL1'})
    .drop_duplicates(['frame', 'track_APPL1', 'track_Lamp1'], ignore_index=True)
)

# Merge collisions data with size data for EEA1
collisions5 = (
    pd.merge(collisions5, tracked2[['r_um', 'track']], 
             left_on='track_Lamp1', right_on='track',
            )
    .drop('track', axis=1)
    .rename(columns={'r_um' : 'r_um_Lamp1'})
    .drop_duplicates(['frame', 'track_APPL1', 'track_Lamp1'], ignore_index=True)
)

## 6.4 Save intermediate results

In [ ]:
# Save filtered results to file
datasets['Filter3'] = {
    'collisions_dict' : collisions3_dict,
    'collisions_func' : collisions4_func,
    'collisions' : collisions5,
}

with open(csv_name, 'wb') as f:
    f.write(pickle.dumps(datasets))

# 7 Compare intermediate results

## 7.1 Load intermediate results

In [ ]:
with open(csv_name, 'rb') as f:
    datasets = pickle.load(f)

# Gather all data into a dict
channel1, channel2 = channels = datasets['Metadata']['channels']
colors = datasets['Metadata']['colors']
movie_list = datasets['Metadata']['movie_list']
frame_rates = datasets['Metadata']['frame_rates']

tracks1 = copy.deepcopy(datasets['Filter2']['tracks1'])
tracks2 = copy.deepcopy(datasets['Filter2']['tracks2'])
movies = copy.deepcopy(datasets['Filter2']['movies'])

tracked1 = copy.deepcopy(datasets['Filter2']['tracked1'])
tracked2 = copy.deepcopy(datasets['Filter2']['tracked2'])
conversions2 = copy.deepcopy(datasets['Filter2']['conversions'])
collisions2 = copy.deepcopy(datasets['Filter2']['collisions'])
conversions3 = copy.deepcopy(datasets['Filter3']['collisions_dict'][1.0])

buckets = copy.deepcopy(datasets['Filter2']['buckets'])
data1 = copy.deepcopy(datasets['Filter2']['data1'])
data2 = copy.deepcopy(datasets['Filter2']['data2'])

## 7.2 Compare size distributions

In [ ]:
data = [tracked1['r_um'].values, tracked2['r_um'].values]

for i, d in enumerate(data):
    data[i] = d[(d > d.min()) & (d < d.max())]
    display(Markdown((f"$\mu[{channels[i]}] = {np.mean(d):0.2f} \pm {np.std(d):0.2f} \mu$m")))

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 2.5))

params = dict(data=data, ax=ax, alpha=0.5, palette=(cyan, magenta), stat='probability', shrink=3)
h = sns.histplot(**params)
ax.set_xlim(0.15, 0.75)
ax.set_xlabel(u"Radius ($\mu$m)")
for t, l in zip(h.legend_.texts, (channel1, channel2)):
    t.set_text(l)

In [ ]:
for index, channel in enumerate(channels):
    df = pd.DataFrame(data[index], columns=[channel])
    export_csv(df, f"Size_{channel}")

In [ ]:
bw_adjust = 5

fig, ax = plt.subplots()
params = dict(ax=ax, bw_adjust=bw_adjust, palette=(magenta, cyan))
sns.kdeplot(data=data[::-1], **params)
ax.set_xlim(0, 1)
ax.set_xlabel(u"Radius ($\mu$m)")
ax.legend(labels=(channel1, channel2))

## 7.3 Comparize collision distances

In [ ]:
data = collisions2['surf_dist'].values, collisions3['surf_dist'].values

fig, ax = plt.subplots()
params = dict(data=data, ax=ax, palette=(gray, gold))
sns.histplot(**params)
ax.set_xlim(-1, 0.2)
ax.set_xlabel(u"Distance ($\mu$m)")

In [ ]:
bw_adjust = 1
cut = 0

fig, ax = plt.subplots()
params = dict(ax=ax, bw_adjust=bw_adjust, cut=cut, palette=(gray, gold))
sns.kdeplot(data=data, **params)

ax.set_xlim(-0.9, 0.1)
ax.set_xlabel(u"Distance ($\mu$m)")

# 8. Visualize final results

## 8.1 Load intermediate results

In [ ]:
# Get proper CSV name (latest export by default)
datapaths = dict()
for path in glob.glob('Dataset*.csv'):
    match = re.match(r'^Dataset_(.*).csv$', path)
    if match:
        name = match.group(1)
        datapaths[name] = path

In [ ]:
# Import all datasets
datasets = dict()
for name, path in datapaths.items():
    with open(path, 'rb') as f:
        dataset = pickle.load(f)
    datasets[name] = dataset

In [ ]:
names = list(datasets.keys())
print(names)

In [ ]:
name = names[0]
dataset = datasets[name]
print(name)

In [ ]:
# Gather all data into a dict
channel1, channel2 = channels = dataset['Metadata']['channels']
colors = dataset['Metadata']['colors']
movie_list = dataset['Metadata']['movie_list']
frame_rates = dataset['Metadata']['frame_rates']

# Set ID and track columns
track_columns = [f'track_{channel}' for channel in channels]
id_columns = track_columns + ['movie']

tracks1 = copy.deepcopy(dataset['Filter2']['tracks1'])
tracks2 = copy.deepcopy(dataset['Filter2']['tracks2'])
movies = copy.deepcopy(dataset['Filter2']['movies'])

tracked1 = copy.deepcopy(dataset['Filter2']['tracked1'])
tracked2 = copy.deepcopy(dataset['Filter2']['tracked2'])
conversions = copy.deepcopy(dataset['Filter2']['conversions'])

buckets = copy.deepcopy(dataset['Filter2']['buckets'])
data1 = copy.deepcopy(dataset['Filter2']['data1'])
data2 = copy.deepcopy(dataset['Filter2']['data2'])

## 8.2 Set desired filters for collisions

In [ ]:
# Choose specific value of furthest distance for collisions
min_separation = 0.5
max_separation = 0.2

def collisions4_func(threshold):
    collisions = copy.deepcopy(dataset['Filter1']['collisions'])
    return collisions[collisions['surf_dist'] < threshold]

collisions = pd.merge(
    pd.merge(
        pd.merge(
            # Remove collisions identified as possible cotracking events
            dataset['Filter2']['collisions'],
            dataset['Filter3']['collisions'],
        ),
        # Remove collisions where greatest distance was not above threshold
        dataset['Filter3']['collisions_dict'][min_separation],
    ),
    # Remove collisions where surfaces were not less than threshold
    collisions4_func(max_separation),
).drop_duplicates(['frame', 'track_APPL1', f'track_{channel2}'])

# Remove collisions where EEA1 may have been larger than expected
collisions = collisions[(collisions[f'r_um_{channel2}'] > tracked2['r_um'].min())]

print(f"min_separation={min_separation}, max_separation={max_separation}: {len(collisions)} remaining ({100 * len(collisions) / len(dataset['Filter1']['collisions']):0.2f}%)")

In [ ]:
(len(collisions), 
 len(collisions) / len(tracked1.drop_duplicates(['movie', 'track'])),
 len(collisions.drop_duplicates(f'track_{channel1}')) / len(tracked1.drop_duplicates(['movie', 'track'])),
)

In [ ]:
# Save data to CSV files for manuscript
export_dir = f"Exports_{name}_{today}"
os.path.exists(export_dir) or os.makedirs(export_dir)

data1.to_csv(f"{export_dir}{os.path.sep}Data_APPL1.csv")
data2.to_csv(f"{export_dir}{os.path.sep}Data_EEA1.csv")
tracked1.to_csv(f"{export_dir}{os.path.sep}Tracks_APPL1.csv")
tracked2.to_csv(f"{export_dir}{os.path.sep}Tracks_EEA1.csv")
collisions.to_csv(f"{export_dir}{os.path.sep}Collisions.csv")
conversions.to_csv(f"{export_dir}{os.path.sep}Conversions.csv")
buckets.to_csv(f"{export_dir}{os.path.sep}Buckets.csv")

In [ ]:
def export_csv(df:pd.DataFrame, filename:str, columns=[], dirname=""):
    if not dirname:
        dirname = f"Exports_{name}_{today}"
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    if len(columns) == 0:
        columns = df.columns
    pathname = os.path.join(dirname, filename+'.csv')
    df[columns].to_csv(pathname)

## 8.3 Separate fusions from conversions

In [ ]:
# Calculate fusion events
fusions = list()
for index, (track1, track2, movie) in tqdm(enumerate(zip(tracks1, tracks2, movies)), total=len(movies)):
    conversion = conversions.groupby(id_columns).get_group((track1, track2, movie)).iloc[0]
    start = conversion['overlap_start'] - conversion['track_'+channel2+'_start']
    prior_failed_conversion = len(conversions.groupby(id_columns).get_group(tuple(conversion[id_columns])).query(f"overlap_start < {conversion['overlap_start']}")) > 1
    if start < 1 or prior_failed_conversion:
        case = 'convert'
    else:
        case = 'fusion'
    case = 'convert' if start < 1 else 'fusion'
    fusion = {f"track_{channel1}" : track1, f'track_{channel2}' : track2, 'start' : start, 'case' : case, 'movie' : movie}
    fusions.append(fusion)
fusions = pd.DataFrame(fusions)

In [ ]:
fusions['case'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.ecdfplot(fusions['start'], ax=ax, stat='proportion')
ax.set_xlabel(f"Frames between Start of {channel2} Track and Start of Conversion")
ax.grid(True)
ax.set_xlim(0, 140)

## 8.4 Separate collisions from conversions

In [ ]:
min_frames_before = 8
min_frames_around = 5

df = list()
n = -1
u = 2
cases = list()
for track1, track2, movie in tqdm(zip(tracks1, tracks2, movies), total=len(tracks1)):
    case = {'track_'+channel1 : track1, 
            'track_'+channel2 : track2, 
            'movie' : movie, 'case' : '', 'c2c' : 0}
    
    # Get relevant signal and collision data
    datum1 = data1.groupby(['track_'+channel1, 'movie']).get_group((track1, movie)).sort_values(by='frame').drop_duplicates(subset='frame')
    conversion = conversions.drop_duplicates().groupby(['track_'+channel1, 'movie']).get_group((track1, movie)).iloc[0]
    
    # Get frames for each event of interest
    start_frame1 = conversion['track_'+channel1+'_start']
    start_frame2 = conversion['track_'+channel2+'_start']
    convert_frame = conversion['overlap_start']
    
    prior_failed_conversion = len(conversions.groupby(id_columns).get_group(tuple(conversion[id_columns])).query(f"overlap_start < {conversion['overlap_start']}")) > 1
    
    try:
        collision = collisions.groupby(['track_'+channel1, 'movie']).get_group((track1, movie)).sort_values(by='frame')
        collision = collision[collision['surf_dist'] <= 0.25]
        #collision = collision[collision['surf_dist'] > -0.25]
        
        collide_frame = collision[collision['frame'] < convert_frame]['frame'].max()
        
        track3 = collision.groupby('frame').get_group(collide_frame).iloc[0]['track_'+channel2]
        datum3 = tracked2.groupby(['track', 'movie']).get_group((track3, movie)).sort_values(by='frame').drop_duplicates(subset='frame')
        
    except (KeyError, ValueError):
        collide_frame = np.nan
        
    if (convert_frame - start_frame1) < min_frames_before:
        # APPL1 tracked for insufficient frames beforehand
        case['case'] = f"not enough data ({channel1})"
        case['c2c'] = np.nan
    elif np.isnan(collide_frame):
        if (convert_frame - start_frame2) <= 0 or prior_failed_conversion:
            # Direct conversion means no collisions detected
            case['case'] = 'conversion, unaided'
        else:
            # Direct fusion means no prior collisions detected
            case['case'] = 'fusion, unaided'
        case['c2c'] = 0
    elif len(datum3) < min_frames_around:# and np.mean(datum3['signal_nrm_'+channel2]) > 5:
        # Colliding EEA1 tracked for insufficient frames
        case['case'] = f"not enough data ({channel2})"
        case['c2c'] = np.nan
    else:
        if (convert_frame - start_frame2) <= 0 or prior_failed_conversion:
            # Collision leading to conversion
            case['case'] = 'conversion, collision-induced'
        else:
            # Collision leading to fusion
            case['case'] = 'fusion, collision-induced'
        case['c2c'] = convert_frame - collide_frame

        # Save intensity traces for analysis of collisions cases
        between = datum1[(datum1['frame'] >= (collide_frame - convert_frame)) & (datum1['frame'] <= 0)]
        between = between[['frame', 'signal_sub_'+channel1, 'signal_sub_'+channel2]].drop_duplicates()

        x, y1, y2 = np.transpose(between.values)
        y1 = y1 / y1[n]
        y2 = y2 / y2[n]

        if all((-u < y1) & (y1 < u) & (-u < y2) & (y2 < u)):
            df.append([{'frame' : x_, 'signal_'+channel1 : y1_, 'signal_'+channel2 : y2_, 'track_'+channel1: track1, 'movie': movie} for x_, y1_, y2_ in zip(x, y1, y2)])
    cases.append(case)

cases = pd.DataFrame(cases).sort_values(by='case')
df = pd.DataFrame(itertools.chain(*df))
df['time'] = df['frame'] * df['movie'].map(frame_rates)

In [ ]:
cases2 = cases[cases['case'].map(lambda x: not x.startswith('not enough data'))]
num_cases = len(cases2)
print(num_cases)

In [ ]:
cases2['case'].value_counts()

In [ ]:
labels = np.sort(cases2['case'].unique())
markers = {label: ('s' if 'direct' in label else 'o') for label in labels}
palette = cyan, gray, magenta, white

def get_label(key):
    if key in ('r_um', ):
        label = key[0] + u" ($\mu$m)"
    elif key in ('V_um', ):
        label = key[0] + u" ($\mu$m$^{3}$)"
    elif key in ('signal_nrm_'+channel1, 'signal_nrm_'+channel2):
        label = key + u" (%)"
    elif key in ('length', ):
        label = key + u" (frames)"
    else:
        label = key
    
    return label

In [ ]:
grouped = data1.groupby(id_columns)
labels = np.sort(cases2['case'].unique())
query = '(frame >= -4) & (frame < -0)'
#query = '(frame >= 4) & (frame < 8)'
results = list()
for case in labels:
    for _, row in cases2.query(f'case == "{case}"').iterrows():
        track1, track2, movie = row[id_columns]
        group = grouped.get_group((track1, track2, movie)).query(query)
        if len(group) > 0:
            values = dict(group.mean(axis=0))
            #result = {'case': 'fusion' if 'fusion' in case else case, **values}
            result = {'case': case, **values}
            results.append(result)
results = pd.DataFrame(results)
len(results)

In [ ]:
results['case'].value_counts()

In [ ]:
data = copy.deepcopy(results)
data['case'].replace('conversion, unaided', ' unaided', inplace=True)
data['case'].replace('conversion, collision-induced', ' collision-induced', inplace=True)
data['case'].replace('fusion, unaided', '  unaided', inplace=True)
data['case'].replace('fusion, collision-induced', '  collision-induced', inplace=True)

In [ ]:
key = 'r_um'
#key = 'signal_sub_'+channel1

fig, ax = plt.subplots(figsize=(8*cm, 5*cm), dpi=150)
params = dict(data=data, x=key, y='case', ax=ax)
sns.boxplot(**params, palette=palette, saturation=1, whis=1)
ax.set_xlabel(get_label(key))
ax.set_ylabel(None)
ax.set_title('conversions     fusions')

In [ ]:
export_csv(data, 'SizeBeforeEvent', ['r_um', 'case'])

In [ ]:
results.query("case == 'conversion, unaided'")['r_um'].values

In [ ]:
key = 'signal_sub_'+channel1

fig, ax = plt.subplots(figsize=(8, 5))
params = dict(data=results, x=key, y='case', ax=ax)
sns.boxplot(**params, palette=palette, saturation=1, whis=1)
sns.stripplot(**params, color='black')
ax.set_xlabel(get_label(key))
ax.set_ylabel(None)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

x = 'signal_sub_'+channel1
#x = 'r_um'
y = 'length'
hue = 'case'
style = 'case'
params = dict(data=results, x=x, y=y, alpha=0.8, hue=hue, ax=ax, markers=markers, style=style, palette=palette)

sns.scatterplot(**params)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel(get_label(x))
ax.set_ylabel(get_label(y))

In [ ]:
print(cases['case'].value_counts())
print()
print(100 * cases2['case'].value_counts().div(num_cases).round(2))

In [ ]:
print(cases['case'].value_counts())
print()
print(100 * cases2['case'].value_counts().div(num_cases).round(2))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.ecdfplot((cases['c2c'] * df['movie'].map(frame_rates)).dropna(), ax=ax, stat='proportion')
ax.set_xlabel('Time between Collision and Start of Conversion (s)')
ax.grid(True)
ax.set_xlim(0, 120)

## 8.5 Visualize intensity time traces

In [ ]:
alpha = 0.05

fig, axs = plt.subplots(2, 1, figsize=(5, 5))
ax1, ax2 = np.ravel(axs)

for _, g in df.groupby(['track_'+channel1, 'movie']):
    x, y1, y2 = np.transpose(g[['time', 'signal_'+channel1, 'signal_'+channel2]].values)
    
    ax1.plot(x, y1, alpha=alpha, color=colors[channel1])
    ax2.plot(x, y2, alpha=alpha, color=colors[channel2])
    
for ax, channel in zip((ax1, ax2), channels):
    ax.set_xlim(-90, 0)
    #ax.set_ylim(-1, 2)
    ax.set_ylim(0, 2)
    if ax == ax2:
        ax.set_xlabel('Time Relative to Start of Conversion (s)')
    ax.set_ylabel('Relative Intensity')
    ax.set_title(channel)

fig.tight_layout()

In [ ]:
alpha = 0.05

fig, ax = plt.subplots(figsize=(5, 3))
  
for channel in channels:
    g = df.groupby('time')['signal_' + channel]
    x, y = zip(*g.mean().items())
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        l, u = zip(*g.apply(get_ci).values)

    ax.plot(x, y, color=colors[channel], label=channel)
    
for _, g in df.groupby(['track_'+channel1, 'movie']):
    x, y1, y2 = np.transpose(g[['time', 'signal_'+channel1, 'signal_'+channel2]].values)
    ax.plot(x, y1, alpha=alpha, color=colors[channel1])
    ax.plot(x, y2, alpha=alpha, color=colors[channel2])
    
ax.set_xlim(-60, 0)
#ax.set_ylim(-1.25, 2.25)
ax.set_ylim(0, 2)
ax.set_xlabel('Time Relative to Start of Conversion (s)')
ax.set_ylabel('Relative Intensity')

ax.legend(loc='upper left')

fig.tight_layout()

In [ ]:
# Collate all trajectories for visualization
meaned = True
normed = True

signals = dict.fromkeys(channels)
for channel, data in zip(channels, (data1, data2)):
    for (movie, event), grouped in data.groupby(['movie', 'event']):
        if 0 not in grouped['frame'].values:
            continue
        
        ints = grouped.filter(like='signal')
        if meaned:
            ints = ints.div(grouped['V_um'].values, axis=0)
        if normed:
            ints = ints.div(ints[(grouped['frame'] == 0)].values, axis=1)
        grouped[ints.columns] = ints
        
        signals[channel] = pd.concat([signals[channel], grouped], ignore_index=True)

In [ ]:
int_type = 'nrm'
sig_columns = ['signal_' + int_type + '_' + c for c in channels]

data = copy.deepcopy(signals[channel1].drop_duplicates(subset=['track_'+channel1, 'frame']))
n1 = [len(group) for frame, group in data.groupby('frame')]
t1 = [group['time'].iloc[0] for frame, group in data.groupby('frame')]

data = copy.deepcopy(df.drop_duplicates(subset=['track_'+channel1, 'frame']))
n2 = [len(group) for frame, group in data.groupby('frame')]
t2 = [group['time'].iloc[0] for frame, group in data.groupby('frame')]

fig, ax = plt.subplots(figsize=(5, 2))
ax.plot(t1, n1, color=cyan)
ax.plot(t2, n2, color=magenta)
ax.set_xlim(-60, 60)
ax.set_xlabel('Time from Start of Cotracking (s)')
ax.set_ylabel('Tracked Count (#)')

In [ ]:
confidence = 0.99
int_type = 'sub'
min_time = 10
max_time = min_time + 10
# max_time = 60

title = f"{min_time} s < Conversion Time $\leq$ {max_time} s"
sig_columns = ['signal_' + int_type + '_' + c for c in channels]

t = list()
M = list()
S = list()
L = list()
U = list()
n = list()

tracks1, tracks2, movies = map(np.asarray, zip(*conversions[['track_'+channel1, 'track_'+channel2, 'movie']].values))
grouped = {c : signals[c].groupby('frame') for c in channels}

for frame in tqdm(range(-50, 50), total=100):
    try:
        groups = {c : grouped[c].get_group(frame) for c in channels}
    except KeyError:
        continue
    
    df = pd.DataFrame()
    for track1, track2, movie in zip(tracks1, tracks2, movies):
        groups2 = dict()
        for c in channels:
            try:
                group2 = groups[c].groupby('movie').get_group(movie)
            except KeyError:
                group2 = pd.DataFrame(columns=groups[c].columns)
            groups2[c] = group2
        
        frame_rate = frame_rates[movie]
        min_frame = int(min_time // frame_rate)
        max_frame = int(max_time // frame_rate)
        
        if frame <= max_frame and track1 in groups2[channel1]['track_'+channel1].values:
            c1 = channel1
            track = track1
        elif track2 in groups2[channel2]['track_'+channel2].values:
            c1 = channel2
            track = track2
        else:
            continue
        
        g = groups2[c1].groupby('track_' + c1).get_group(track)
        if (g['length'].iloc[0] > min_frame) and (g['length'].iloc[0] <= max_frame):            
            df = pd.concat((df, g.filter(like=int_type)))
        else:
            continue
    
    df = df[(df > 0.25) & (df <= 4.0)].dropna().reset_index(drop=True)
    
    t.append(frame * frame_rate)
    M.append(dict(df.mean()))
    S.append(dict(df.std()))
    n.append(len(df))

    if frame != 0 and len(df) > 1:
        l, u = np.transpose([get_ci(df[c2], confidence) for c2 in sig_columns])
    else:
        l = u = (np.nan, np.nan)

    L.append(dict(zip(sig_columns, l)))
    U.append(dict(zip(sig_columns, u)))

t = np.asarray(t)
n = np.asarray(n)

M = pd.DataFrame(M)
S = pd.DataFrame(S)
L = pd.DataFrame(L)
U = pd.DataFrame(U)

In [ ]:
fig, ax = plt.subplots(figsize=(12*cm, 3*cm), dpi=150)

for channel in channels:
    column = 'signal_' + int_type + '_' + channel
    m = M[column]
    s = S[column]
    l = L[column]
    u = U[column]
    
    ax.axvline(0, color='k', alpha=0.5, ls='--', lw=1.0)
    ax.axvline(min_time, color='gray', alpha=0.5, ls='--', lw=1.0)
    ax.axvline(max_time, color='gray', alpha=0.5, ls='--', lw=1.0)
    ax.axvspan(min_time, max_time, facecolor='gray', alpha=0.05)
    
    ax.plot(t, m, color=colors[channel], label=channel)
    ax.fill_between(t, l, u, alpha=0.2, color=colors[channel])

ax.set_xlim(-20, +60)
ax.set_ylim(0.75, 1.45)
ax.set_xlabel(u"Time Relative to Start of Conversion (s)")
ax.set_ylabel(u"Relative Intensity")
#ax.set_title(title)
#ax.set_title('WT')
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
df = pd.DataFrame({
    **pd.DataFrame(t, columns=['time_rel']).to_dict(),
    **M.rename(columns={'signal_sub_EEA1' : 'mean_EEA1', f'signal_sub_{channel1}' : f'mean_{channel1}'}),
    **U.rename(columns={'signal_sub_EEA1' : 'upper_EEA1', f'signal_sub_{channel1}' : f'upper_{channel1}'}),
    **L.rename(columns={'signal_sub_EEA1' : 'lower_EEA1', f'signal_sub_{channel1}' : f'lower_{channel1}'}),
})

export_csv(df.query("time_rel >= -22.5 and time_rel <= 62.5"), f"IntensityTrace_{min_time:d}-{max_time:d}")

In [ ]:
try:
    ts
except NameError:
    ts = dict()
    Ms = dict()
    Ss = dict()
    Ls = dict()
    Us = dict()

ts[min_frame] = t
Ms[min_frame] = M
Ss[min_frame] = S
Ls[min_frame] = L
Us[min_frame] = U

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12*cm, 14*cm), dpi=150)
axs = np.ravel(axs)

for ax, channel in zip(axs, channels):
    ax.axvline(0, color='k', alpha=0.5, ls='--', lw=1.0)
    column = 'signal_' + int_type + '_' + channel
    for min_frame, color in zip(Ms.keys(), (magenta, cyan, gray)):
        t = ts[min_frame]#[indexes]
        m = Ms[min_frame][column]#[indexes]
        s = Ss[min_frame][column]#[indexes]
        l = Ls[min_frame][column]#[indexes]
        u = Us[min_frame][column]#[indexes]
        
        min_time = int(min_frame * frame_rate)
        max_time = int(min_frame * frame_rate) + 10
        en_dash = u"\u2013"
        label = f"{min_time}{en_dash}{max_time} s"
        
        ax.plot(t, m, color=color, label=label)
        ax.fill_between(t, l, u, alpha=0.1, color=color)
    
    ax.set_xlim(-20, +60)
    ax.set_ylim(0.7, 1.7)
    ax.set_ylabel(u"Relative Intensity")
    if ax == axs[1]:
        ax.set_xlabel(u"Time Relative to Start of Conversion (s)")
    ax.set_title(channel, {'fontweight' : 'medium'})

fig.tight_layout()

## 8.6 Compare across conditions

In [ ]:
try:
    tC
except NameError:
    tC = dict()
    MC = dict()
    SC = dict()
    LC = dict()
    UC = dict()

tC[condition] = t
MC[condition] = M
SC[condition] = S
LC[condition] = L
UC[condition] = U

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12*cm, 14*cm), dpi=150)
axs = np.ravel(axs)

for ax, condition in zip(axs, tC.keys()):
    ax.axvline(0, color='k', alpha=0.5, ls='--', lw=1.0)
    for channel, color in zip(channels, (cyan, magenta)):
        column = 'signal_' + int_type + '_' + channel
        
        t = tC[condition]
        m = MC[condition][column]
        s = SC[condition][column]
        l = LC[condition][column]
        u = UC[condition][column]
        
        min_time = int(min_frame * frame_rate)
        max_time = int(min_frame * frame_rate) + 10
        en_dash = u"\u2013"
        label = f"{min_time}{en_dash}{max_time} s"
        
        ax.plot(t, m, color=color, label=channel)
        ax.fill_between(t, l, u, alpha=0.1, color=color)
    
    ax.set_xlim(-20, +60)
    ax.set_ylim(0.7, 1.7)
    ax.set_ylabel(u"Relative Intensity")
    if ax == axs[1]:
        ax.set_xlabel(u"Time Relative to Start of Conversion (s)")
    ax.set_title('WT' if condition == '_Senthil' else condition.lower(), {'fontweight' : 'medium'})

fig.tight_layout()

In [ ]:
plt.hist(cases.dropna()['c2c'])

In [ ]:
cases2 = cases.dropna().reset_index(drop=True)

## 8.7 Create heatmap of collisions and conversions

In [ ]:
cases_ = cases2[cases2['case'].map(lambda x: x.startswith('fusion'))]

In [ ]:
data_ = list()
for index, row in tqdm(cases_.iterrows(), total=len(cases_.drop_duplicates())):
    track1, track2, movie = row[['track_'+channel1, 'track_'+channel2, 'movie']]
    
    try:
        # Get correct conversion event
        conversion = (conversions[conversions['movie'] == movie]
                      .query(f"track_{channel1} == {track1} and track_{channel2} == {track2}")
                      .sort_values(by='overlap_stop')
                      .drop_duplicates(keep='last')
                     ).iloc[-1]
        convert_start = conversion['overlap_start']

        # Get number of collisions before start of conversion
        collision = (collisions[collisions['movie'] == movie]
                     .query(f"track_{channel1} == {track1}")
                     .drop_duplicates()
                    )
        collision = collision[collision['frame'] <= convert_start]
        collision = collision[collision['surf_dist'] <= 0.2]
        num_collisions = len(collision)
    except Exception as e:
        print(e)
        num_collisions = 0
    
    frames_before = conversion['overlap_start'] - conversion['track_'+channel1+'_start']
    frames_during = conversion['overlap_stop'] - conversion['overlap_start']

    data_.append({'num_collisions' : num_collisions,
                  'frames_before' : frames_before,
                  'frames_during' : frames_during,
                  'time_before': frames_before * frame_rates[movie],
                  'time_during' : frames_during * frame_rates[movie],
                 })

data_ = pd.DataFrame(data_).drop_duplicates()
data_['num_collisions'].value_counts().iloc[:10]

In [ ]:
# Set minimum time required before conversion to be analyzed
min_time_before = 28
data = data_[data_['time_before'] >= min_time_before]

base = 10
data['block_before'] = (data['time_before'] // base).astype(int)
data['block_during'] = (data['time_during'] // base).astype(int)

# Set minimum conversion length
min_collisions = 0
min_convert_len = 4 / base

# Set maximum conversion length
max_collisions = max(data['num_collisions'])
max_convert_len = 135 / base

# Create matrix to be used for generating heatmap
mat = np.zeros((max(data['block_during']) + 1, max(data['num_collisions']) + 1), dtype=int)
for (num_collision, block_during), count in data.groupby(['num_collisions', 'block_during']).size().items():
    mat[block_during, num_collision] = count

# Optionally remove cases of less-than minimum collisions
mat[:, :min_collisions] = 0

In [ ]:
N = max(5 * (mat.max() // 5 + 1), 5)
if N > 5:
    cmap = LinearSegmentedColormap.from_list('custom', [(0, white), (0.2, gray), (0.3, cyan), (1, magenta)], N=N, gamma=1.0)
else:
    cmap = LinearSegmentedColormap.from_list('custom', [(0, white), (0.5, gray), (0.6, cyan), (1, magenta)], N=N, gamma=1.0)

frame_rate = 2.5
yticks = np.arange(0, max_convert_len) * base # frame_rate

yticklabels = list()
for ytick in yticks:
    if np.mod(ytick, 1) == 0:
        yticklabel = str(round(ytick))
    else:
        yticklabel = ''
    yticklabels.append(yticklabel)

xticklabels = [(str(v) if v % 1 == 0 else '') for v in range(100)]

fig, ax = plt.subplots(figsize=(8*cm, 8*cm), dpi=150)
sns.heatmap(mat, cmap=cmap, cbar=True, square=True, xticklabels=xticklabels, yticklabels=yticklabels, vmin=-0.5, vmax=N-0.5)
ax.set_xlabel('Collisions before\nConversion (#)')
ax.set_ylabel('Conversion Duration (s)')
ax.set_xlim(min_collisions, min_collisions + 6)
ax.set_ylim(min_convert_len, max_convert_len - min_convert_len)

ax.collections[0].set_clim(0, N)

In [ ]:
df = pd.DataFrame(mat[1:12, :])
df /= df.sum().sum()
export_csv(df, 'CollisionsToEvents')

# 9 Check final analysis statistics

## 9.1 Find all collisions leading to conversions and/or fusions

In [ ]:
max_separation = 0.25

converted_indexes = list()
for index, row in tqdm(cases2.iterrows(), total=len(cases2.drop_duplicates())):
    track1, track2, movie = row[['track_'+channel1, 'track_'+channel2, 'movie']]
    
    try:
        # Get correct conversion event
        conversion = (conversions
                      .query(f"movie == {movie} and track_{channel1} == {track1} and track_EEA1 == {track2}")
                      .sort_values(by='overlap_stop')
                      .drop_duplicates(keep='last')
                     ).iloc[-1]
        convert_start = conversion['overlap_start']

        # Get indexes of collisions before start of conversion
        collision = (collisions
                     .query(f"movie == {movie} and track_{channel1} == {track1}")
                     .drop_duplicates()
                    )
        collision = collision[(collision['frame'] < convert_start) & 
                              (collision['surf_dist'] < max_separation)]
        collision = collision.sort_values('frame')
        if len(collision)> 0:
            converted_indexes.append(collision.index[-1])
    except KeyError as e:
        print(e)

In [ ]:
# Look only at collisions leading directly to conversions
collisions_ = collisions.loc[converted_indexes]

## 9.2 Calculate distribution of nearest-neighbor distances

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
spatial_columns = ['x_um', 'y_um', 'z_um']

# Construct the KDTree
def make_tree(df, columns=spatial_columns):
    centroids = df[columns].values
    tree = cKDTree(centroids)
    return tree

In [ ]:
# Create dict of tracked data for convenience
data_dict = dict()
for channel, df in zip(channels, (tracked1, tracked2)):
    data = {(movie, frame) : group for (movie, frame), group in df.groupby(['movie', 'frame'])}
    data_dict[channel] = data

In [ ]:
# Get all combinations of movies and frames
movie_frames = [v for v in data_dict[channel1].keys() if v in data_dict[channel2].keys()]

In [ ]:
# Calculate KDTrees for each channel and timepoint
trees_dict = dict()
for channel, data in data_dict.items():
    trees = {(movie, frame) : make_tree(group) for (movie, frame), group in data.items()}
    trees_dict[channel] = trees

In [ ]:
# Query the KDTree for nearest neighbor distances (same species)
cent_dists_dict = dict()
for channel in channels:
    cent_dists = dict()
    for (movie, frame), tree in tqdm(trees_dict[channel].items()):
        centroids = data_dict[channel][movie, frame][spatial_columns].values
        distances, _ = tree.query(centroids, k=2)
        cent_dists[movie, frame] = distances[:, 1]
    cent_dists_dict[channel] = cent_dists

In [ ]:
# Use `cdist` to calculate nearest neighbor distances (opposite species)
from scipy.spatial.distance import cdist

exclude_conversions = False #True
focuson_conversions = True #False

channel1, channel2 = channels
keys = f"{channel1}-{channel1}", f"{channel2}-{channel2}", f"{channel1}-{channel2}"

min_cent_dists_dict = dict()
min_surf_dists_dict = dict()
for key in keys:
    print(key)
    match = re.match(r"(\w*)-(\w*)", key)
    key1, key2 = match.group(1), match.group(2)
    
    min_cent_dists = dict()
    min_surf_dists = dict()
    for (movie, frame) in tqdm(movie_frames):
        # Get identities of all in-progress conversions
        converting = conversions.query(f"movie == {movie} and overlap_start <= {frame} and overlap_stop >= {frame}")
        if len(converting) > 0:
            converting_tracks1, converting_tracks2 = zip(*converting.filter(regex=r"^track_(APPL1|EEA1)$").values)
        else:
            converting_tracks1, converting_tracks2 = tuple(), tuple()
        
        # Get data for current movie and frame
        data1 = data_dict[key1][movie, frame]
        data2 = data_dict[key2][movie, frame]
        
        if exclude_conversions:
            data1 = data1.query(f"track not in {converting_tracks1}")
            data2 = data2.query(f"track not in {converting_tracks2}")
        
        if focuson_conversions:
            data1 = data1.query(f"track in {converting_tracks1}")
            data2 = data2.query(f"track in {converting_tracks2}")
        
        # Get all particle positions
        cents1 = data1[spatial_columns].values
        cents2 = data2[spatial_columns].values

        # Calculate the distances between each point of vector 1 and vector 2
        cent_dists = cdist(cents1, cents2)
        if key1 == key2:
            np.fill_diagonal(cent_dists, np.nan)
        
        try:
            min_indexes = np.nanargmin(cent_dists, axis=1)
        except ValueError:
            continue

        # Calculate minimum centroid and surface distances
        min_cent_dists[movie, frame] = list()
        min_surf_dists[movie, frame] = list()
        for i, j in enumerate(min_indexes):
            min_cent_dist = cent_dists[i, j]
            if min_cent_dist < 1e-6:
                min_surf_dist = np.nan
            else:
                min_surf_dist = min_cent_dist - (data1.iloc[i]['r_um'] + data2.iloc[j]['r_um'])
            min_cent_dists[movie, frame].append(min_cent_dist)
            min_surf_dists[movie, frame].append(min_surf_dist)

    min_cent_dists_dict[key] = min_cent_dists
    min_surf_dists_dict[key] = min_surf_dists

# Concatenate all results together
min_cent_dist = {k : np.hstack(list(v.values())) for k, v in min_cent_dists_dict.items()}
min_surf_dist = {k : np.hstack(list(v.values())) for k, v in min_surf_dists_dict.items()}

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.1
binrange = -1, 5

fig, ax = plt.subplots(1, figsize=(2.5, 2.5))
for key, color in zip(keys, (cyan, magenta, gold)):
    sns.histplot(
        data=min_surf_dist[key],
        binwidth=binwidth,
        binrange=binrange,
        stat='count',
        ax=ax,
        alpha=0.5,
        color=color,
    )

ax.set_xlim(binrange)
ax.set_ylim(0, 750)
ax.set_xlabel(r"Nearest Distance ($\mu$m)")
ax.set_ylabel("Count")
ax.legend(keys)

In [ ]:
if not exclude_conversions and focuson_conversions:
    min_surf_dist_converting = copy.deepcopy(min_surf_dist)
elif exclude_conversions and not focuson_conversions:
    min_surf_dist_nonconverting = copy.deepcopy(min_surf_dist)

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.1
binrange = -1, 5

fig, ax = plt.subplots(1, figsize=(2.5, 2.5))
for data, color in zip((min_surf_dist_nonconverting['APPL1-EEA1'], min_surf_dist_converting['APPL1-EEA1']), ("#EEEEEE", "#111111")):
    sns.histplot(
        data=data,
        binwidth=binwidth,
        binrange=binrange,
        stat='count',
        ax=ax,
        alpha=0.5,
        color=color,
    )

ax.set_xlim(binrange)
ax.set_ylim(0, 25000)
ax.set_xlabel(r"Nearest Distance ($\mu$m)")
ax.set_ylabel("Count")
#ax.set_title("APPL1-EEA1 Nearest Neighbours")
ax.legend(("No", "Yes"), title="Converting?")

In [ ]:
# Plot final results for all combinations (smoothed histogram)
bw_adjust = 0.5
binrange = -1, 5

fig, ax = plt.subplots(1)
for key, color in zip(keys, (cyan, magenta, gold)):
    sns.kdeplot(
        data=min_surf_dist[key],
        bw_adjust=bw_adjust,
        ax=ax,
        color=color,
    )

ax.set_xlim(binrange)
ax.set_xlabel(r"Nearest Distance ($\mu$m)")
ax.set_ylabel("Density")
ax.legend(keys)

## 9.3 Calculate local endosome densities

In [ ]:
# Define the range of search radii
search_radii = range(1, 11)

local_densities = dict()
for search_radius in tqdm(search_radii):
    search_volume = (4 / 3) * np.pi * search_radius ** 3
    
    # Calculate local density of endosomes in each channel
    densities_dict = dict()
    for channel in channels:
        densities = dict()
        for (movie, frame) in movie_frames:
            positions = data_dict[channel][movie, frame][spatial_columns]

            # Construct the KDTree using the object positions
            tree = cKDTree(positions)

            # Calculate the local density for all objects
            neighbor_indices = tree.query_ball_point(positions, search_radius)
            
            # Count number of nearest neighbors within search radius
            neighbor_counts = np.asarray([len(v)-1 for v in neighbor_indices])
            
            # Calculate the local density as the count of neighbors within the search radius
            local_density = neighbor_counts / search_volume
            
            densities[movie, frame] = local_density

        densities_dict[channel] = densities

    # Concatenate all results together
    local_density = {k : np.hstack(list(v.values())) for k, v in densities_dict.items()}
    local_densities[search_radius] = local_density

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10, 20))
axs = np.ravel(axs)

for ax_index, (search_radius, local_density) in enumerate(local_densities.items()):
    ax = axs[ax_index]
    
    # Plot final results for all combinations (histogram)
    binwidth = 0.02
    binrange = 0, 0.5
    
    for channel, color in zip(channels, (cyan, magenta)):
        sns.histplot(
            data=local_density[channel],
            binwidth=binwidth,
            binrange=binrange,
            stat='density',
            ax=ax,
            alpha=0.5,
            color=color,
        )

    ax.set_xlim(binrange)
    ax.set_xlabel(r"Local Density ($\mu$m$^{-3}$)")
    ax.set_ylabel("PDF")
    ax.set_title(fr"Search Radius = {search_radius:0.1f} $\mu$m")
    ax.legend(channels)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 2.5))
axs = np.ravel(axs)

search_radius = 3
local_density = local_densities[search_radius]

# Plot final results for all combinations (histogram)
binwidth = 0.02
binrange = 0, 0.5

for channel, color in zip(channels, (cyan, magenta)):
    sns.histplot(
        data=local_density[channel],
        binwidth=binwidth,
        binrange=binrange,
        stat='probability',
        ax=ax,
        alpha=0.5,
        color=color,
    )

ax.set_xlim(binrange)
ax.set_xlabel(r"Local Density ($\mu$m$^{-3}$)")
ax.legend(channels)

In [ ]:
for channel in channels:
    df = pd.DataFrame(local_densities[3][channel], columns=[channel])
    export_csv(df, f"LocalDensity_{channel}")

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10, 20))
axs = np.ravel(axs)

for ax_index, (search_radius, local_density) in enumerate(local_densities.items()):
    ax = axs[ax_index]
    
    # Plot final results for all combinations (smoothed histogram)
    bw_adjust = 2.5
    binrange = 0, 0.5

    for channel, color in zip(channels, (cyan, magenta)):
        sns.kdeplot(
            data=local_density[channel],
            bw_adjust=bw_adjust,
            ax=ax,
            color=color,
        )

    ax.set_xlim(binrange)
    ax.set_xlabel(r"Endosome Density ($\mu$m$^{-3}$)")
    ax.set_ylabel("Density")
    ax.set_title(fr"Search Radius = {search_radius:0.1f} $\mu$m")
    ax.legend(channels)

fig.tight_layout()

## 9.4 Calculate endosome velocity distributions (moving average)

In [ ]:
window_size = 5

moving_dict = dict.fromkeys(channels)
for channel, data in zip(channels, (tracked1, tracked2)):
    moving = {k : list() for k in set(movies)}
    for (movie, track), group in tqdm(data.groupby(['movie', 'track'])):
        if len(group) < window_size * 2:
            continue

        displacements = (group
                         .filter(regex='(x|y|z)_um')
                         .diff(axis=0)
                        )
        time_steps = (group[['frame']] * frame_rates[movie]).diff()
        velocities = (displacements / time_steps.values).dropna()

        velocities_moving = velocities.rolling(window_size).mean().dropna()
        moving[movie].extend(list(np.sqrt(np.sum(velocities_moving ** 2, axis=1)).values))
    moving_dict[channel] = moving

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.0075
binrange = 0, 0.25

fig, ax = plt.subplots(1, figsize=(4, 2.5))
for channel, data in moving_dict.items():
    sns.histplot(
        data=np.hstack(list(data.values())),
        binwidth=binwidth,
        binrange=binrange,
        stat='probability',
        ax=ax,
        alpha=0.5,
        color=colors[channel],
    )

ax.set_xlim(binrange)
ax.set_xlabel('Velocity ($\mu$m s$^{-1}$)')
ax.legend(channels)

In [ ]:
for channel in channels:
    df = pd.DataFrame(np.hstack(list(moving_dict[channel].values())), columns=[channel])
    export_csv(df, f"Velocity_{channel}")

In [ ]:
# Plot final results for all combinations (histogram)
binrange = 0, 0.25

fig, ax = plt.subplots(1)
for channel, data in moving_dict.items():
    sns.kdeplot(
        data=np.hstack(list(data.values())),
        ax=ax,
        color=colors[channel],
    )

ax.set_xlim(binrange)
ax.set_xlabel('Velocity ($\mu$m s$^{-1}$)')
ax.legend(channels)

In [ ]:
{k : np.std(np.hstack(list(v.values()))) for k, v in moving_dict.items()}

## 9.5 Calculate endosome step size distributions (instantaneous)

In [ ]:
step_size_dict = dict.fromkeys(channels)
for channel, data in zip(channels, (tracked1, tracked2)):
    step_sizes = {k : list() for k in set(movies)}
    for (movie, track), group in tqdm(data.groupby(['movie', 'track'])):
        if len(group) < window_size * 2:
            continue

        displacements = (group
                         .filter(regex='(x|y|z)_um')
                         .diff(axis=0)
                        )
        frame_steps = group[['frame']].diff()
        step_sizes[movie].extend(list(np.sqrt(np.sum((displacements / frame_steps.values).dropna() ** 2, axis=1)).values))
    step_size_dict[channel] = step_sizes

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.1
binrange = 0, 1.5

fig, ax = plt.subplots(1)
for channel, data in step_size_dict.items():
    sns.histplot(
        data=np.hstack(list(data.values())),
        binwidth=binwidth,
        binrange=binrange,
        stat='count',
        ax=ax,
        alpha=0.5,
        color=colors[channel],
    )

ax.set_xlim(binrange)
ax.set_xlabel('Instantaneous Step Sizes ($\mu$m frame$^{-1}$)')
ax.legend(channels)

In [ ]:
# Plot final results for all combinations (histogram)
binrange = 0, 1.5
bw_adjust = 3

fig, ax = plt.subplots(1)
for channel, data in step_size_dict.items():
    sns.kdeplot(
        data=np.hstack(list(data.values())),
        bw_adjust=bw_adjust,
        ax=ax,
        color=colors[channel],
    )

ax.set_xlim(binrange)
ax.set_xlabel('Instantaneous Step Sizes ($\mu$m frame$^{-1}$)')
ax.legend(channels)

In [ ]:
# Instantaneous step sizes
{k : np.mean(np.hstack(list(v.values()))) for k, v in step_size_dict.items()}

In [ ]:
# Instantaneous velocities
{k : np.mean(np.hstack([(np.array(b) * frame_rates[a]) for a, b in v.items()])) for k, v in moving_dict.items()}

In [ ]:
# Moving average velocities
{k : np.mean(np.hstack(list(v.values()))) for k, v in moving_dict.items()}

## 9.6 Calculate expected collision frequencies

In [ ]:
# Calculate collision cross-section of APPL1 and EEA1 endosomes
radius1 = tracked1.groupby('movie')['r_um'].mean()
radius2 = tracked2.groupby('movie')['r_um'].mean()
sigma_ = np.pi * (radius1 + radius2) ** 2

In [ ]:
search_radius = 5
search_volume = (4 / 3) * np.pi * search_radius ** 3

# Calculate local density of endosomes in each movie and channel
densities_dict = dict()
for channel in channels:
    densities = list()
    for (movie, frame) in tqdm(movie_frames):
        positions = data_dict[channel][movie, frame][spatial_columns]
        
        # Construct the KDTree using the object positions
        tree = cKDTree(positions)
        
        # Calculate the local density for all objects
        neighbor_indices = tree.query_ball_point(positions, search_radius)
        
        # Count number of nearest neighbors within search radius
        neighbor_counts = np.asarray([len(v)-1 for v in neighbor_indices])
        
        # Calculate the local density as the count of neighbors within the search radius
        local_density = neighbor_counts / search_volume
        
        # Gather all results into DataFrame
        df = data_dict[channel][movie, frame][['movie', 'frame', 'track']]
        df['density'] = local_density
        
        # Append Dataframe
        densities.append(df)

    densities_dict[channel] = pd.concat(densities)

In [ ]:
# Calculate local densities of all collision events
colliding_densities_dict = dict()
for channel in channels:
    colliding_densities_dict[channel] = pd.merge(
        densities_dict[channel],
        collisions[['movie', 'frame', f'track_{channel}']],
        left_on=['movie', 'frame', 'track'],
        right_on=['movie', 'frame', f'track_{channel}'],
    )

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.01
binrange = 0, 0.25

fig, axs = plt.subplots(1, 2, figsize=(13, 5))
axs = np.ravel(axs)
for ax, channel in zip(axs, channels):
    for index, color in enumerate(('gray', 'gold')):
        if index == 0:
            data = densities_dict[channel]['density']
        elif index == 1:
            data = colliding_densities_dict[channel]['density']
        
        sns.histplot(
            data=data,
            binwidth=binwidth,
            binrange=binrange,
            stat='density',
            ax=ax,
            alpha=0.5,
            color=color,
        )

    ax.set_xlim(binrange)
    ax.set_xlabel(r"Local Density ($\mu$m$^{-3}$)")
    ax.set_title(fr"{channel}: Search Radius = {search_radius:0.1f} $\mu$m")
    ax.legend(('all', 'colliding'))

fig.tight_layout()

In [ ]:
# Calculate local densities of all conversion events
converting_densities_dict = dict()
for channel in channels:
    converting_densities_dict[channel] = pd.merge(
        densities_dict[channel],
        conversions[['movie', 'overlap_start', f'track_{channel}']],
        left_on=['movie', 'frame', 'track'],
        right_on=['movie', 'overlap_start', f'track_{channel}'],
    )

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 0.01
binrange = 0, 0.25

fig, axs = plt.subplots(1, 2, figsize=(13, 5))
axs = np.ravel(axs)
for ax, channel in zip(axs, channels):
    for index, color in enumerate(('gray', 'gold')):
        if index == 0:
            data = densities_dict[channel]['density']
        elif index == 1:
            data = converting_densities_dict[channel]['density']
        
        sns.histplot(
            data=data,
            binwidth=binwidth,
            binrange=binrange,
            stat='density',
            ax=ax,
            alpha=0.5,
            color=color,
        )

    ax.set_xlim(binrange)
    ax.set_xlabel(r"Local Density ($\mu$m$^{-3}$)")
    ax.set_title(fr"{channel}: Search Radius = {search_radius:0.1f} $\mu$m")
    ax.legend(('all', 'converting'))

fig.tight_layout()

In [ ]:
# Calculate the average number density (of both particles)
rho_1 = pd.Series({k : v for k, v in densities_dict['APPL1'].groupby('movie')['density'].mean().items()}, name='rho_')
rho_1.index.name = 'movie'
rho_2 = pd.Series({k : v for k, v in densities_dict['EEA1'].groupby('movie')['density'].mean().items()}, name='rho_')
rho_2.index.name = 'movie'

In [ ]:
np.std(rho_1), np.std(rho_2)

In [ ]:
# Calculate the mean free path of APPL1 and EEA1 endosomes
lambda_1 = 1 / (np.sqrt(2) * rho_1 * sigma_)
lambda_2 = 1 / (np.sqrt(2) * rho_2 * sigma_)

In [ ]:
np.mean(lambda_1), np.mean(lambda_2)

In [ ]:
# Calculate the average displacements (of both particles)
disp_1 = pd.Series({k : np.mean(v) for k, v in step_size_dict['APPL1'].items()}, name='displacement')
disp_1.index.name = 'movie'
disp_2 = pd.Series({k : np.mean(v) for k, v in step_size_dict['EEA1'].items()}, name='displacement')
disp_2.index.name = 'movie'

In [ ]:
# Calculate average time between collisions
np.mean(lambda_1 / (disp_1 / pd.Series(frame_rates))), np.mean(lambda_2 / (disp_2 / pd.Series(frame_rates)))

In [ ]:
# Calculate the collision frequency of APPL1 and EEA1 endosomes
v_rms1 = pd.Series({k : np.mean(v) / frame_rates[k] for k, v in step_size_dict['APPL1'].items()}, name='v_rms')
v_rms1.index.name = 'movie'
v_rms2 = pd.Series({k : np.mean(v) / frame_rates[k] for k, v in step_size_dict['EEA1'].items()}, name='v_rms')
v_rms2.index.name = 'movie'

nu_1 = v_rms1 / lambda_1
nu_2 = v_rms2 / lambda_2

In [ ]:
# Average collision frequency of APPL1 and EEA1 endosomes
np.mean(nu_1), np.mean(nu_2)

In [ ]:
# Average frame-frame step size of APPL1 endosomes
np.mean(v_rms1 * pd.Series(frame_rates))

In [ ]:
# Average frame-frame step size of EEA1 endosomes
np.mean(v_rms2 * pd.Series(frame_rates))

In [ ]:
counts1 = dict()
for movie, group in tracked1.groupby('movie'):
    counts1[movie] = group['frame'].value_counts().values[:50]
counts1 = pd.Series({k : np.mean(v) for k, v in counts1.items()}, name='counts')
counts1.index.name = 'movie'

counts2 = dict()
for movie, group in tracked2.groupby('movie'):
    counts2[movie] = group['frame'].value_counts().values[:50]
counts2 = pd.Series({k : np.mean(v) for k, v in counts2.items()}, name='counts')
counts2.index.name = 'movie'

In [ ]:
np.mean(counts1), np.mean(counts2)

In [ ]:
# Estimate number of collisions per frame
fig, ax = plt.subplots()
ax.hist(nu_1 * counts1 * pd.Series(frame_rates), density=True)
ax.set_xlabel("Collisions per Frame")
ax.set_ylabel("Density")

In [ ]:
# Calculate average number of APPL1 collisions onto EEA1 per unit time (frame)
np.mean(nu_1 * counts1 * pd.Series(frame_rates))

In [ ]:
# Calculate average number of EEA1 collisions onto APPL1 per unit time (frame)
np.mean(nu_2 * counts2 * pd.Series(frame_rates))

In [ ]:
# Calculate average number of confirmed conversions per unit time
np.mean(conversions.groupby('movie')['movie'].count() / tracked1[['movie', 'frame']].drop_duplicates().groupby('movie')['frame'].count()) / frame_rate

In [ ]:
# Estimate number of collisions per frame
fig, ax = plt.subplots()
ax.hist(nu_2 * counts2 * pd.Series(frame_rates), density=True)
ax.set_xlabel("Collisions per Frame")
ax.set_ylabel("Density")

## 9.7 Calculate empirical distribution of times between collisions

In [ ]:
min_seconds = 240
min_surf_dist = -0.25
max_surf_dist = +0.0

collision_intervals_dict = dict()
for channel, tracked in zip(channels, (tracked1, tracked2)):
    collision_intervals = list()
    collisions_ = collisions.query(f"surf_dist > {min_surf_dist} and surf_dist < {max_surf_dist}")
    for (movie, track), group in tqdm(collisions_.groupby(['movie', f'track_{channel}'])):
        min_frames = min_seconds / frame_rates[movie]
        if len(tracked.query(f"movie == {movie} and track == {track}")) < min_frames:
            continue
        if len(group) > 1:
            collision_interval = (group['frame']
                                  .sort_values()
                                  .diff()
                                  .dropna()
                                  .values
                                 ) * frame_rates[movie]
        else:
            collision_interval = [min_seconds]
        collision_intervals.extend(collision_interval)
    collision_intervals_dict[channel] = np.asarray(collision_intervals)

In [ ]:
{k : np.mean(v[v < min_seconds]) for k, v in collision_intervals_dict.items()}

In [ ]:
# Plot final results for all combinations (histogram)
binwidth = 10
binrange = 0, min_seconds

fig, ax = plt.subplots(1, figsize=(4, 2.5))
for channel, data in collision_intervals_dict.items():
    sns.histplot(
        data=data[data < min_seconds],
        binwidth=binwidth,
        binrange=binrange,
        stat='probability',
        ax=ax,
        alpha=0.5,
        color=colors[channel],
    )

ax.set_xlim(binrange)
ax.set_xlabel('Time Between Collisions (s)')
ax.legend(channels)

In [ ]:
for channel in channels:
    data = collision_intervals_dict[channel]
    df = pd.DataFrame(data[data < min_seconds], columns=[channel])
    export_csv(df, f"MeanCollisionTime_{channel}")